In [156]:
# generate data
import Distributions
using Random

Random.seed!(42)

n_samples = 10
dimensionality = 2
rθ = [-0.5, 0.2]
inputs = randn(n_samples, dimensionality)
σ(w, x) = 1/(1+exp(-w'x))
πs = [σ(rθ, inputs[i, :]) for i in 1:n_samples]
outputs = [rand(Distributions.Bernoulli(πs[i])) for i in 1:n_samples]

10-element Vector{Bool}:
 1
 1
 1
 1
 0
 1
 1
 0
 1
 1

In [165]:
# test nonlinear
using ForneyLab
import ForneyLab: unsafeMean, unsafeCov

graph = FactorGraph()

T = n_samples
x = Vector{Variable}(undef, T)
y = Vector{Variable}(undef, T)

# Hearing aid parameters
@RV θ  ~ GaussianMeanPrecision(placeholder(:m_θ, dims=(order,)), placeholder(:W_θ, dims=(order, order)))
f(w,x) = 1/(1+exp(-w'x))
for i in 1:T
    @eval $(Symbol("func$i"))(θ) = f(θ,inputs[$i, :])
    @RV x[i] ~ Nonlinear{Sampling}(θ, g=eval(Symbol("func$i")), in_variates=[Multivariate], out_variate=Univariate)
    @RV y[i] ~ Bernoulli(x[i])
    placeholder(y[i], :y, index=i)
end

In [166]:
algo = messagePassingAlgorithm(θ, free_energy=true)
src_code = algorithmSourceCode(algo, free_energy=true);

In [167]:
println(src_code);

begin

function step!(data::Dict, marginals::Dict=Dict(), messages::Vector{Message}=Array{Message}(undef, 58))

messages[1] = ruleSPGaussianMeanPrecisionOutNPP(nothing, Message(Multivariate, PointMass, m=data[:m_θ]), Message(MatrixVariate, PointMass, m=data[:W_θ]))
messages[2] = ruleSPBernoulliIn1PN(Message(Univariate, PointMass, m=data[:y][10]), nothing)
messages[3] = ruleSPNonlinearSIn1MN(func10, messages[2], nothing, variate=Multivariate)
messages[4] = ruleSPBernoulliIn1PN(Message(Univariate, PointMass, m=data[:y][9]), nothing)
messages[5] = ruleSPNonlinearSIn1MN(func9, messages[4], nothing, variate=Multivariate)
messages[6] = ruleSPEqualityFn(messages[5], nothing, messages[3])
messages[7] = ruleSPBernoulliIn1PN(Message(Univariate, PointMass, m=data[:y][8]), nothing)
messages[8] = ruleSPNonlinearSIn1MN(func8, messages[7], nothing, variate=Multivariate)
messages[9] = ruleSPEqualityFn(messages[8], nothing, messages[6])
messages[10] = ruleSPBernoulliIn1PN(Message(Univariate, PointMass,

In [168]:
# Load algorithm
eval(Meta.parse(src_code))

freeEnergy (generic function with 1 method)

In [169]:
import ProgressMeter

data = Dict(:y => outputs, :m_θ => zeros(dimensionality), :W_θ => 0.1*diageye(dimensionality))
marginals = step!(data)

Dict{Any, Any} with 11 entries:
  :x_3  => SampleList(s=[0.79, 0.65, 0.51, 0.36, 0.53, 0.62, 0.61, 0.47, 0.43, …
  :x_10 => SampleList(s=[0.01, 0.02, 0.05, 1.08e-03, 1.17e-03, 0.10, 4.91e-03, …
  :x_2  => SampleList(s=[0.71, 0.66, 0.61, 0.55, 0.60, 0.67, 0.62, 0.60, 0.50, …
  :x_5  => SampleList(s=[0.07, 0.74, 0.20, 0.13, 0.06, 0.10, 0.03, 0.46, 0.25, …
  :x_1  => SampleList(s=[0.87, 0.59, 0.83, 0.75, 0.96, 0.86, 0.75, 0.62, 0.96, …
  :x_4  => SampleList(s=[0.58, 0.93, 0.85, 0.84, 0.58, 0.97, 0.74, 0.91, 0.83, …
  :x_9  => SampleList(s=[0.87, 0.80, 0.83, 1.00, 0.55, 0.98, 0.06, 0.68, 1.00, …
  :θ    => 𝒩(m=[-1.33, 0.81], w=[[1.08, -0.08][-0.08, 1.36]])…
  :x_6  => SampleList(s=[0.94, 0.31, 0.57, 0.52, 0.39, 0.03, 0.74, 0.06, 0.04, …
  :x_7  => SampleList(s=[0.46, 0.66, 0.65, 0.81, 0.42, 0.79, 0.85, 0.67, 0.40, …
  :x_8  => SampleList(s=[0.41, 0.62, 0.13, 0.23, 0.52, 0.23, 0.41, 0.12, 0.03, …

In [170]:
meθ = unsafeMean(marginals[:θ])

2-element Vector{Float64}:
 -1.3319520146225503
  0.8149393727598759

In [171]:
weθ = unsafeCov(marginals[:θ])

2×2 Matrix{Float64}:
 0.929187   0.0559767
 0.0559767  0.737735

In [172]:
println("training errors = $(sum([round(f(meθ, inputs[i, :])) - round(πs[i]) for i in 1:n_samples]))")

training errors = 0.0


In [173]:
# test nonlinear
using ForneyLab
import ForneyLab: unsafeMean, unsafeCov

graph = FactorGraph()

T = n_samples
x = Vector{Variable}(undef, T)
z = Vector{Variable}(undef, T)
y = Vector{Variable}(undef, T)

# Hearing aid parameters
@RV θ  ~ GaussianMeanPrecision(placeholder(:m_θ, dims=(order,)), placeholder(:W_θ, dims=(order, order)))
f(w,x) = 1/(1+exp(-w'x))
for i in 1:T
#     @eval $(Symbol("func$i"))(θ) = f(θ,inputs[$i, :])
    @RV z[i] ~ GaussianMeanPrecision(x, 1e3*diageye(dimensionality))
#     @RV x[i] ~ Nonlinear{Sampling}(θ, g=eval(Symbol("func$i")), in_variates=[Multivariate], out_variate=Univariate)
    @RV x[i] ~ Nonlinear{Sampling}(θ, z[i], g=f, in_variates=[Multivariate, Multivariate], out_variate=Univariate)
    @RV y[i] ~ Bernoulli(x[i])
    placeholder(y[i], :y, index=i)
end